In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
# veri kaynağı: http://veribilimi.co/data/iris.csv
df = pd.read_csv("D:\\OneDrive\\egitim\\verilen\\bogazici-compec-python-ml\\Datasets\\iris.csv")

# Veri keşfi

In [3]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
X = df.iloc[:,:-1].values
X.shape

(150, 4)

In [5]:
y = df.iloc[:,-1].values
y.shape

(150,)

# Kategorik Nitelik Dönüştürme

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
labelencoder = LabelEncoder()

In [8]:
X[:,-1] = labelencoder.fit_transform(X[:,0])

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=142)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112, 4)
(38, 4)
(112,)
(38,)


# Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Model Creation and Training

In [14]:
from sklearn.linear_model import LogisticRegression

In [24]:
classifier = LogisticRegression(random_state=142, solver='lbfgs', multi_class='auto')

# Computing cross-validated metrics

In [25]:
from sklearn.model_selection import cross_val_score

In [26]:
scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='accuracy')

In [27]:
scores

array([0.95833333, 0.82608696, 0.95652174, 0.85714286, 0.95238095])

In [28]:
scores.mean()

0.9100931677018634

In [29]:
 print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.9101 (+/- 0.1136)


# Optimal hiperparametreleri bulma

In [30]:
# Az önce yukarıda kullandığımız sınıflandırıcının parametreleri
classifier.get_params

<bound method BaseEstimator.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=142, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)>

In [27]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

In [33]:
from sklearn.model_selection import GridSearchCV

In [34]:
param_grid = [
  {
    'C': [1.0, 10.0, 100.0,1000.0], 
   'tol': [1e-4, 1e-6],
      'solver':['newton-cg', 'lbfgs', 'sag'],
      'penalty':['l2'],
      'max_iter':[10,100,300]
    },
    {
     'C': [1.0, 10.0, 100.0,1000.0], 
   'tol': [1e-4, 1e-6],
      'solver':['liblinear','saga'],
      'penalty':['l1'],
        'max_iter':[10,100,300]
    }
 ]

In [53]:
grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=param_grid,
                          scoring='accuracy',
                          cv=4,
                          n_jobs=4)
# maksimum için n_jobs=-1 kullanılır.

In [54]:
grid_search = grid_search.fit(X_train, y_train)

D:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [55]:
best_score = grid_search.best_score_
best_score

0.9910714285714286

In [56]:
best_params = grid_search.best_params_
best_params

{'C': 100.0,
 'max_iter': 10,
 'penalty': 'l2',
 'solver': 'newton-cg',
 'tol': 0.0001}

In [57]:
best_estimator = grid_search.best_estimator_
best_estimator

LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=142, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

# Model kaydetme

In [58]:
from sklearn.externals import joblib

In [59]:
joblib.dump(best_estimator, "best_estimator_log_reg_iris.pkl")

['best_estimator_log_reg_iris.pkl']

# Model Okuma

In [60]:
best_estimator_loaded = joblib.load("best_estimator_log_reg_iris.pkl")

In [71]:
best_estimator_loaded.get_params

<bound method BaseEstimator.get_params of LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=142, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)>

# Modeli tüm veri setiyle tekrar eğitme

In [85]:
# En iyi hiperparametrelerin ne olduğunu öğrendik. Artık onları kullanarak en iyi modeli
# eğitebiliriz.

In [72]:
classifier2 = LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=142, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [73]:
classifier2.fit(X_train,y_train)

D:\Anaconda\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=142, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

# Predictions

In [74]:
y_pred = classifier2.predict(X_test)

## Tahmin sonuçlarını numpy.ndarray'den dataframe yapma (İncelemek daha Kolay)

In [75]:
predictions = pd.DataFrame(data=y_pred,    # values
                index=range(len(y_pred)),    # 1st column as index
                   columns=['y_pred'])  # 1st row as the column names

# Sadece y_pred'den oluşan df'e test(gerçek) y_test'i sütun olarak ekleme
predictions['y_test'] = y_test
predictions.head()

,y_pred,y_test
0,Iris-setosa,Iris-setosa
1,Iris-versicolor,Iris-versicolor
2,Iris-versicolor,Iris-versicolor
3,Iris-virginica,Iris-virginica
4,Iris-versicolor,Iris-versicolor


# Hata Matrisi (Confusion Matrix)

In [76]:
from sklearn.metrics import confusion_matrix

In [77]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(cm)

[[ 7  0  0]
 [ 0 15  2]
 [ 0  2 12]]


In [78]:
from sklearn.metrics import accuracy_score

In [79]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      89.47


# Elle Accuracy hesaplama

In [80]:
errors = predictions[predictions['y_pred'] != predictions['y_test']]
errors.head()

,y_pred,y_test
8,Iris-versicolor,Iris-virginica
14,Iris-virginica,Iris-versicolor
18,Iris-versicolor,Iris-virginica
19,Iris-virginica,Iris-versicolor


In [81]:
predictions[predictions['y_pred'] != predictions['y_test']].count()

y_pred    4
y_test    4
dtype: int64

In [82]:
len(y_test)

38

In [83]:
accuracy = (len(y_test) - len(errors)) / len(y_test)
accuracy

0.8947368421052632